In [1]:
using CSV, DataFrames, JuMP, Gurobi, LinearAlgebra, Random

In [2]:
# Constants

nP = 10 # number of people
nT = 210 # number of timeslots in a week
# Organized in 30 minute interals from 9am-12pm, Sunday-Saturday
# 30 intervals/day * 7 days/week
cM = 1e7 # multiplier for the minimal matching objective

print("")

In [3]:
# Load Data

# 0-1 matrix of whether person can mentor/mentee at time T
load_A = Matrix(CSV.read("schedules.csv", DataFrame, header=0))[3:end, 2:end]  # nP x nT
A = zeros(nP, nT)
for i in 1:nP, j in 1:nT
    A[i, j] = parse(Int64, load_A[i, j])
end


# for these, what if someone is only a mentor or only a mentee? would it be zero?
aR = Matrix(CSV.read("num_sessions.csv", DataFrame))[:, 2] # nP x 1: max num of session person can mentor
# aE = Matrix(CSV.read("max_menteeing.csv", DataFrame)) # nP x 1: max num of session person can mentee (can we remove this?)
r  = Matrix(CSV.read("rank.csv", DataFrame, header=0))[:, 2]# nP x 1, rank

# Don't we just need one per person? Lessens the amount of data, plus not everyone is a mentor or mentee
p = Matrix(CSV.read("poomsae_pref.csv", DataFrame))[:, 2]
s = Matrix(CSV.read("sparring_pref.csv", DataFrame))[:, 2]
# pR = Matrix(CSV.read("mentor_poomsae.csv", DataFrame)) # nP x 1: 0-1 does mentor prefer poomsae 
# pE = Matrix(CSV.read("mentee_poomsae.csv", DataFrame)) # nP x 1: 0-1 does mentee prefer poomsae
# sR = Matrix(CSV.read("mentor_sparring.csv", DataFrame)) # nP x 1: 0-1 does mentor prefer sparring
# sE = Matrix(CSV.read("mentee_sparring.csv")) # nP x 1: 0-1 does mentee prefer sparring

print("")

In [4]:
# Create Cost Matrix

alpha = 1
beta  = 1

C = zeros(nP, nP)

# Adjusting to only have one set of data for sparring and poomsae preferences
for i = 1:nP, j = 1:nP
    C[i, j] += alpha * (r[i]-r[j]-2+(r[j] >= 9))
    C[i, j] -= beta  * (p[i]*p[j]+s[i]*s[j]-p[i]*s[i]-p[j]*s[j])
    C[i, j] -= 1
end

# for i = 1:nP, j = 1:nP
#     C[i, j] += alpha * (r[i]-r[j]-2+(r[j] >= 9))
#     C[i, j] -= beta  * (pR[i]*pE[j]+sR[i]*sE[j]-pR[i]*sR[i]-pE[j]*sE[j])
#     C[i, j] -= 1
# end


In [5]:
# Rank Constraint Matrix

R = zeros(nP, nP, nT)

for i = 1:nP, j = 1:nP, T = 1:nT
    if r[i] >= r[j]+2 || (r[i] >= r[j]+1 && r[j] >= 9)  # Maybe make this lower
        R[i, j, T] = 1
    end
end

In [6]:
m = Model(Gurobi.Optimizer)
# set_optimizer_attribute(m, "OutputFlag", 0)

print("")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-03


In [7]:
@variable(m, 0 <= x[1:nP, 1:nP, 1:nT] <= 1)
@variable(m, 0 <= t[1:nP] <= 1, Int)

print("")

In [8]:
# Rank Constraint
@constraint(m, x .<= R)

# Maximum Constraints
for P = 1:nP
    @constraint(m, sum(x[i, P, T] for i=1:nP, T = 1:nT) <= aR[P])
    @constraint(m, sum(x[P, i, T] for i=1:nP, T = 1:nT) <= 1) # replaced with ones
end

# Availability Constraints
# for i = 1:nP, j = i+1:nP, T=1:nT
#     @constraint(m, x[i,j,T] <= A[i,T]*A[j,T])
# end

# Single Mentor Constraint
for i = 1:nP, j = i+1:nP, k = 1:nP, T=1:nT
    @constraint(m, x[i,k,T]+x[j,k,T] <= 1)
end

# Making t work
for P = 1:nP
    @constraint(m, sum(x[i,P,T] for i=1:nP, T=1:nT) >= t[P])
    @constraint(m, 0.01*sum(x[i,P,T] for i=1:nP, T=1:nT) <= t[P])
end

print("")

In [9]:
@objective(m, Min, cM*sum((1-t[P]) for P=1:nP) + sum(x[i,j,T]*C[i,j] for i=1:nP, j=1:nP, T=1:nT))
print("")

In [10]:
optimize!(m)

objective_value(m)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i7-6950X CPU @ 3.00GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 115540 rows, 21010 columns and 294020 nonzeros
Model fingerprint: 0x159427ef
Variable types: 21000 continuous, 10 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+00, 1e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 1.000000e+08
Presolve removed 115540 rows and 21010 columns
Presolve time: 0.12s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.14 seconds (0.30 work units)
Thread count was 1 (of 20 available processors)

Solution count 2: 8e+07 1e+08 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.999999900000e+07, best bound 7.999999900000e+07, gap 0.0000%

User-callback calls 370, 

7.9999999e7

In [11]:
count = 0
x_sol = value.(x)
for i in 1:nP, j in 1:nP, k in 1:nT
    if x_sol[i, j, k] != 0
        count += 1
    end
end
@show count

count = 1260


1260